<a href="https://colab.research.google.com/github/Ankitarora2/TechnoCulture/blob/main/AMIESignatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [2]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [11]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

INFO 02-26 12:17:29 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:39 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:49 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:59 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:18:09 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapp

In [12]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1708949943,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-da2e39ff6119457a829dd7b8a1c141d6","object":"model_permission","created":1708949943,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [13]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [14]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

In [15]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of Germany?")

Prediction(
    answer='Berlin'
)

# **DSPY.SIGNATURES**

In [53]:
class GMCPQ(dspy.Signature):
  """ How would you rate the doctor at the following criteria, on a 5 point scale. Poor Less than Satisfactory Satisfactory Good Very Good Cannot rate / Does not apply"""

conversation= dspy.InputField(desc="conversation between doctor and patient")
#context= dspy.InputField(desc="Criteria: Being polite")
ratings= dspy.OutputField(desc="""ratings of doctor on criteria: being polite""")


In [ ]:
Being polite;
Making you feel at ease;
 Listening to you;
Assessing your medical condition;
Explaining your condition and treatment;
Involving you in decisions about your treatment;
Providing or arranging treatment for you

In [54]:
convo="""Patient – Good afternoon, Doctor.

Doctor – Good afternoon, Mr. Bose. How are you?

Patient – I’m doing good, doctor, but my daughter isn’t doing well. Everywhere, people are getting affected with COVID and I am really worried about her.

Doctor – Please have a seat and tell me what happened.

Patient – Last week, my daughter came back from Pune as her college was closed on account of COVID. From the second day, she has had high fever and has been coughing badly. I think that she has contracted the virus on her way home.

Doctor – Okay, I understand your concern. Having a fever and cough doesn’t necessarily mean that someone has contracted the virus. These are symptoms of common cold too. The change in the temperature of the atmosphere could have triggered these symptoms. Still, to put your worries to rest, I am prescribing some medicines and an RT PCR test. Do the test by tomorrow, and if the test results are positive, make sure she is isolated. On the other hand, if the result is negative, just give her the medicine and ask her to drink a lot of water. Also, bring her in so I could examine her.

Patient – Okay, doctor. I will bring her in the evening. Thank you.

Doctor – You are welcome."""

In [56]:
judge = dspy.ChainOfThought(signature=GMCPQ, )
judge(conversation=convo)

ValueError: not enough values to unpack (expected at least 1, got 0)

In [ ]:
class search_retrieval(dspy.Signature):
  """What are the specific patient demographics/symptoms/management plan for the condition"""

condition = dspy.InputField(desc=“medical condition”)
passage=dspy.OutputField(desc=”a passage including specific patient demographics/symptoms/management plan”)


#Search Retrieval

`What are the specific patient demographics/symptoms/management plan for the condition [Condition]?`

In [ ]:
"""What are the specific patient demographics/symptoms/management plan for the condition?"""
#INPUT
condition = dspy.InputField(desc="medical condition")

#OUTPUT
passage=dspy.OutputField(desc="a passage including specific patient demographics/symptoms/management plan")

#Passage Filtering Template



```
For the clinical condition, [Condition], is the following a good description of common demographics/symptoms/management plans (Yes/No)?
Description: [Retrieved Passage]
Answer (Yes/No):
```



In [ ]:
"""For the clinical condition, is the following a good description of common demographics/symptoms/management plans (Yes/No)?"""
#INPUT
condition = dspy.InputField(desc="medical condition")
passage=dspy.InputField(desc="a passage including specific patient demographics/symptoms/management plan")

#OUTPUT
is_good = dspy.OutputField(desc="Yes/No")

#Vignette Generation Template




```
The following are several passages about the demographics, symptoms, and management plan for a given condition. Generate 2 different patient vignettes consistent with these passages. Follow the format of the given example (just list N/A if a particular field is unavailable).
Condition: [Condition]
Demographic Passages: [Retrieved Demographic Passages]
Symptoms Passages: [Retrieved Symptom Passages]
Management Plan Passages: [Retrieved Management Plan Passages]
Example Format: [Oneshot example]
Patient Vignettes for [Condition]:
```



In [ ]:
"""The following are several passages about the demographics, symptoms, and management plan for a given condition. Generate 2 different patient vignettes consistent with these passages. Follow the format of the given example (just list N/A if a particular field is unavailable)"""
#INPUT
condition = dspy.InputField(desc="medical condition")

demographic_passages= dspy.InputField(desc="[Retrieved Demographic Passages]")

symptom_passages= dspy.InputField(desc="[Retrieved Symptom Passages]")

management_passages= dspy.InputField(desc="[Retrieved Management Passages]")

example = dspy.InputField(desc="[Oneshot example]")

#OUTPUT
vignette=dspy.OutputField(desc="2 different patient Vignettes for [MedicalCondition]:)")


# Patient Agent



```
You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. <patient vignette> Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.
```



In [ ]:
"""You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. <patient vignette> Respond to the doctor's questions honestly as they interview you, asking any questions that may come up."""

#INPUT
vignette=dspy.InputField(desc="patient vignette")

doctors_question=dspy.InputField(desc="doctor's question")

#OUTPUT
patients_response=dspy.OutputField(desc="Response to the doctor's questions, along with any questions that may come up.")


#ALTERNATIVE OUTPUT
patients_response=dspy.OutputField(desc="Response to the doctor's questions.")
patients_question=dspy.OutputField(desc="Any questions that may come up.")

# Doctor Agent

`You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.`

In [ ]:
"""You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."""

#INPUT
patients_response=dspy.InputField(desc="patient's response")

#OUTPUT
doctors_response=dspy.OutputField(desc="Response to the patient's questions, along with any questions that may come up.")


#ALTERNATIVE OUTPUT
response=dspy.OutputField(desc="Response to the patient's questions.")
doctors_question=dspy.OutputField(desc="Any questions that may come up. Do not ask more than 2 questions.")


#ASSERT SUGGEST
"""Respond to the patient with a single-turn response to better understand their history and symptoms.
Do not ask more than two questions."""

Self Play Critic

In [ ]:
""" Act as a critic aware of the ground truth diagnosis, to provide in-context feedback to the doctor agent for selfimprovement which can enhance its performance"""
#INPUT

ground_truth=dspy.InputField(desc="ground truth")
preeceding_conversation=dspy.InputField(desc="preeceding conversation")
doctors_response=dspy.InputField(desc="doctor's response")

#OUTPUT
empathy = dspy.OutputField(desc="""Rating(0 to 5) of the doctor's response based on if the doctor agent exhibits empathy and professionalism while addressing the patient’s latest questions or comments in a concise manner.
0: Complete lack of empathy and professionalism, responses are dismissive or unhelpful.
1: Minimal empathy and professionalism, responses are brief and lack warmth or understanding.
2: Some empathy and professionalism, responses are moderately concise but lack depth or personalization.
3: Good empathy and professionalism, responses are sufficiently concise and demonstrate understanding.
4: High level of empathy and professionalism, responses are very concise yet convey genuine concern and understanding.
5: Excellent empathy and professionalism, responses are exceptionally concise, demonstrating deep empathy and professionalism.""")


repititve_questions=dspy.OutputField(desc="""The doctor agent avoids asking too many or repetitive questions (about information already acquired), focusing on a maximum of one or two questions per response.(Yes/No)""")

